# Ensemble learning for image classification (CIFAR100)

### Summary

Test

Group number and member names:

In [1]:
GROUP = "36"
NAME1 = "Timothy Hellberg"
NAME2 = "Lars Liberg"

## 0. Imports

In [2]:
# YOUR CODE HERE
# For dealing with files
import os, sys
import shutil

# For using regex expressions
import re

# For splitting the data
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

# Packages for defining the architecture of our model
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Input, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import regularizers
from keras.optimizers import Adam
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model 
from keras.wrappers.scikit_learn import KerasClassifier

# For generating data
from keras.preprocessing.image import ImageDataGenerator

# One-hot encoding
from keras.utils import np_utils

# Callbacks for training
from keras.callbacks import TensorBoard, EarlyStopping

# Ploting
import matplotlib.pyplot as plt
%matplotlib inline

# Ndarray computations
import numpy as np

# Confusion matrix for assessment step
from sklearn.metrics import confusion_matrix

# Dataset
from keras.datasets import cifar10, cifar100

# Other
import pandas as pd
from subprocess import check_output
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
from time import time
from time import sleep
from scipy import ndimage
from scipy.misc import imresize
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout, Maximum
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Concatenate
from keras import backend as K 

Using TensorFlow backend.


## 1. Loading the data and preprocessing

In [3]:
home_path = "/home/student/deep-machine-learning/Project"
os.chdir(home_path)

# Dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

y_train = to_categorical(y_train, num_classes=100)
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape,                                                                
                                                                                            x_test.shape, y_test.shape))
input_shape = x_train[0,:,:,:].shape
print("input_shape:",input_shape)
model_input = Input(shape=input_shape)

#plt.imshow(x_train[np.random.randint(50000)]);


x_train shape: (50000, 32, 32, 3) | y_train shape: (50000, 100)
x_test shape : (10000, 32, 32, 3) | y_test shape : (10000, 1)
input_shape: (32, 32, 3)


In [4]:
#indices = np.random.randint(50000, size=50000)
#x_train_bag = x_train[indices,:,:,:]
#y_train_bag = y_train[indices,:]
#print(indices)
#print(x_train[[1,3,1,3],:,:,1])
#rint(y_train[[1,3,1,3],:])


## 2. Training

### 2.1 Functions, learners and combiners

In [5]:
# Universal functions

def compile_and_train(model, num_epochs): 
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = home_path + '/weights/CIFAR100/' + model.name + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    return history

def compile_and_train_bagging(model, num_epochs, x_train, y_train): 
    indices = np.random.randint(50000, size=50000)
    x_train_bag = x_train[indices,:,:,:]
    y_train_bag = y_train[indices,:]
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = home_path + '/weights/CIFAR100/' + model.name + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train_bag, y=y_train_bag, batch_size=256, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    del x_train_bag, y_train_bag
    return history

def evaluate_test_accuracy(model):
    pred = model.predict(x_test, batch_size = 256)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    test_accuracy = np.sum(np.equal(pred, y_test)) / y_test.shape[0]  
    return test_accuracy

In [6]:
# Custom base learners

def base_cnn(model_input):
    x = Conv2D(128, (5, 5), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(100, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_base_cnn')
    return model

def base_cnn_2(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(100, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_base_cnn_2')
    return model

def base_cnn_2_conv(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(100, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_base_cnn_2_conv')
    return model

def base_cnn_3_conv(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(100, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_base_cnn_3_conv')
    return model

def conv_pool_cnn(model_input):
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding =    'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(100, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_conv_pool_cnn')
    return model

def all_cnn(model_input):
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(100, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_all_cnn')
    return model

def base_learner_bagging(model_input, num):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(100, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR100_base_learner_' + str(num) + '_bagging')
    return model


In [7]:
# Custom combiners

def average_ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    #print(np.shape(np.array(outputs)))
    #print(outputs[0])
    y = Average()(outputs)
    model = Model(models[0].input, y, name='CIFAR100_average_ensemble')
    return model

def concatenate_ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    print(np.shape(np.array(outputs)))
    y = Concatenate(axis=-1)(outputs)
    model = Model(model_input, y, name='CIFAR100_concatenate_ensemble')
    return model

def basic_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(100, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR100_basic_stacking_' + str(num) + '_ensemble')
    return model

def small_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(100, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR100_small_stacking_' + str(num) + '_ensemble')
    return model

def large_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(100, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR100_large_stacking_' + str(num) + '_ensemble')
    return model

def deep_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(100, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR100_deep_stacking_' + str(num) + '_ensemble')
    return model

def bn_deep_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(100, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR100_bn_deep_stacking_' + str(num) + '_ensemble')
    return model

### 2.2 Train base learners

#### 2.3 Bagging

In [8]:
# Train bagged model
base_model = base_learner_bagging(model_input, 3)
print(base_model.summary())

models = []
for i in np.arange(40,50):
    base_model = base_learner_bagging(model_input, i)
    _ = compile_and_train_bagging(base_model, 10, x_train, y_train)
    models.append(base_model)
    base_model.save_weights('weights_CIFAR100/weights_CIFAR100_base_learner_' + str(i) + '_bagging.hdf5')
    del base_model


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 128)       512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
__________

In [8]:
# Train single model with full dataset
base_model_full = base_learner_bagging(model_input, 50)
_ = compile_and_train(base_model_full, 20)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 13s 335us/step - loss: 3.5066 - acc: 0.1961 - val_loss: 3.4549 - val_acc: 0.1899
Epoch 2/20
40000/40000 [==============================] - 12s 289us/step - loss: 2.5594 - acc: 0.3678 - val_loss: 2.9144 - val_acc: 0.3037
Epoch 3/20
40000/40000 [==============================] - 12s 288us/step - loss: 2.0668 - acc: 0.4753 - val_loss: 2.8751 - val_acc: 0.3190
Epoch 4/20
40000/40000 [==============================] - 11s 287us/step - loss: 1.6615 - acc: 0.5690 - val_loss: 2.8513 - val_acc: 0.3407
Epoch 5/20
40000/40000 [==============================] - 12s 288us/step - loss: 1.2736 - acc: 0.6701 - val_loss: 2.6704 - val_acc: 0.3680
Epoch 6/20
40000/40000 [==============================] - 12s 288us/step - loss: 0.9077 - acc: 0.7697 - val_loss: 2.8693 - val_acc: 0.3623
Epoch 7/20
40000/40000 [==============================] - 12s 288us/step - loss: 0.5882 - acc: 0.8594 - val_loss: 2.

In [ ]:
# Load bagged model (whole models)

models = []
for i in np.arange(0,50):
    models.append(load_model('models/CIFAR100_base_learner_' + str(i) + '_bagging.h5'))
    print(i)

In [8]:
# Load bagged model (only weights)

models_new = []
for i in np.arange(0,50):
    base_model = base_learner_bagging(model_input, i)
    base_model.load_weights('weights_CIFAR100/weights_CIFAR100_base_learner_' + str(i) + '_bagging.hdf5')
    models_new.append(base_model)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [10]:
# Freeze weights in bagged model

model_0 = models_new[2]
model_0.summary()

for model in models_new:
    for layer in model.layers:
        layer.trainable = False
        
model_0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 128)       147584    
_________________________________________________________________
batch_normalization_8 (Batch (None, 13, 13, 128)       512       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 128)         0         
__________

In [ ]:
# Evaluate the different magged models

#base_model = base_learner_bagging(model_input, 3)
#base_model.load_weights('weights/CIFAR10/CIFAR10_base_learner_3_bagging.17-0.75.hdf5')
for model in models:
        print(evaluate_test_accuracy(model))


In [11]:
# Evaluate single model with full dataset
base_model = base_learner_bagging(model_input, 50)
base_model.load_weights('weights/CIFAR100/CIFAR100_base_learner_50_bagging.13-0.39.hdf5')
print(evaluate_test_accuracy(base_model))

0.397


### 2.3 Evaluate simple combiners

In [9]:
# Max voting

max_voting_accuracies = np.zeros((51,1))
for k in np.arange(2,50):
    n_models = k
    models_short = models_new[0:n_models]

    pred_new = np.zeros((10000,100))
    max_voting_model = concatenate_ensemble(models_short, model_input)
    pred = max_voting_model.predict(x_test, batch_size = 256)
    pred = np.expand_dims(pred, axis=1)
    for i in range(n_models):
        j = i*100
        #print(i)
        #print(np.shape(np.array(pred)))
        #print(np.shape(np.array(pred)))
        pred_part = pred[:,:,j:j+100]
        #print(np.shape(np.array(pred)))
        index = np.argmax(pred_part, axis=2)
        #print(np.shape(np.array(index)))
        index = to_categorical(index, num_classes=100)
        #print(np.shape(np.array(index)))
        #print(index[i,:])
        pred_new += index
        #print(pred_new[0:3,:])

    pred_new = np.expand_dims(pred_new, axis=1)
    pred_new = np.argmax(pred_new, axis=2)
    max_voting_accuracies[k] = np.sum(np.equal(pred_new, y_test)) / y_test.shape[0]
    print(k,"test_acc:", max_voting_accuracies[k])

(2,)
2 test_acc: [0.3434]
(3,)
3 test_acc: [0.3872]
(4,)
4 test_acc: [0.4119]
(5,)
5 test_acc: [0.431]
(6,)
6 test_acc: [0.4435]
(7,)
7 test_acc: [0.4508]
(8,)
8 test_acc: [0.4607]
(9,)
9 test_acc: [0.4671]
(10,)
10 test_acc: [0.4742]
(11,)
11 test_acc: [0.4766]
(12,)
12 test_acc: [0.4795]
(13,)
13 test_acc: [0.4849]
(14,)
14 test_acc: [0.4864]
(15,)
15 test_acc: [0.4878]
(16,)
16 test_acc: [0.4905]
(17,)
17 test_acc: [0.4927]
(18,)
18 test_acc: [0.4938]
(19,)
19 test_acc: [0.4928]
(20,)
20 test_acc: [0.4957]
(21,)
21 test_acc: [0.4971]
(22,)
22 test_acc: [0.4989]
(23,)
23 test_acc: [0.4987]
(24,)
24 test_acc: [0.5001]
(25,)
25 test_acc: [0.503]
(26,)
26 test_acc: [0.5028]
(27,)
27 test_acc: [0.5042]
(28,)
28 test_acc: [0.5049]
(29,)
29 test_acc: [0.506]
(30,)
30 test_acc: [0.5057]
(31,)
31 test_acc: [0.5079]
(32,)
32 test_acc: [0.5064]
(33,)
33 test_acc: [0.5062]
(34,)
34 test_acc: [0.5059]
(35,)
35 test_acc: [0.5065]
(36,)
36 test_acc: [0.5081]
(37,)
37 test_acc: [0.5087]
(38,)
38 te

ResourceExhaustedError: OOM when allocating tensor with shape[128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: batch_normalization_139/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, _class=["loc:@batch_normalization_139/cond/Switch_1"], data_format="NCHW", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_93/Relu, batch_normalization_139/gamma/read, batch_normalization_139/beta/read, batch_normalization_1/Const_4, batch_normalization_1/Const_4)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: concatenate_46/concat/_12075 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8152_concatenate_46/concat", tensor_type=DT_FLOAT, _device="/jo

In [10]:
np.savetxt('CIFAR100_max_voting_accuracies.txt', max_voting_accuracies, fmt='%.4f')

In [10]:
np.savetxt('CIFAR100_average_accuracies.txt', average_accuracies, fmt='%.4f')

In [9]:
# Average

average_accuracies = np.zeros((51,1))
#for model in models_new:
#    print(model.name)
#    model.save_weights('weights_CIFAR10/weights_' + model.name +'.hdf5')

for i in np.arange(2,50):
    n_models = i
    #print(n_models)
    models_short = models_new[0:n_models]
    average_model = average_ensemble(models_short, model_input)
    average_accuracies[i] = evaluate_test_accuracy(average_model)
    print(i,"test_acc:", average_accuracies[i])

(2,)
2 test_acc: [0.3939]
(3,)
3 test_acc: [0.4267]
(4,)
4 test_acc: [0.442]
(5,)
5 test_acc: [0.4589]
(6,)
6 test_acc: [0.4681]
(7,)
7 test_acc: [0.4722]
(8,)
8 test_acc: [0.4801]
(9,)
9 test_acc: [0.4847]
(10,)
10 test_acc: [0.4901]
(11,)
11 test_acc: [0.4928]
(12,)
12 test_acc: [0.4972]
(13,)
13 test_acc: [0.498]
(14,)
14 test_acc: [0.4997]
(15,)
15 test_acc: [0.5012]
(16,)
16 test_acc: [0.5016]
(17,)
17 test_acc: [0.504]
(18,)
18 test_acc: [0.503]
(19,)
19 test_acc: [0.502]
(20,)
20 test_acc: [0.5054]
(21,)
21 test_acc: [0.507]
(22,)
22 test_acc: [0.5087]
(23,)
23 test_acc: [0.508]
(24,)
24 test_acc: [0.5083]
(25,)
25 test_acc: [0.5094]
(26,)
26 test_acc: [0.5091]
(27,)
27 test_acc: [0.5103]
(28,)
28 test_acc: [0.5108]
(29,)
29 test_acc: [0.5122]
(30,)
30 test_acc: [0.5104]
(31,)
31 test_acc: [0.5107]
(32,)
32 test_acc: [0.5108]
(33,)
33 test_acc: [0.5109]
(34,)
34 test_acc: [0.512]
(35,)
35 test_acc: [0.5121]
(36,)
36 test_acc: [0.512]
(37,)
37 test_acc: [0.5119]
(38,)
38 test_acc

ResourceExhaustedError: OOM when allocating tensor with shape[128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: batch_normalization_139/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, _class=["loc:@batch_normalization_139/cond/Switch_1"], data_format="NCHW", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_93/Relu, batch_normalization_139/gamma/read, batch_normalization_139/beta/read, batch_normalization_1/Const_4, batch_normalization_1/Const_4)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: average_46/truediv/_12075 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_8198_average_46/truediv", tensor_type=DT_FLOAT, _device="/job:loca

### 2.4 Train stacking combiners

In [12]:
np.savetxt('CIFAR100_basic_stacking_accuracies.txt', basic_stacking_accuracies, fmt='%.4f')

In [16]:
# Train single ensemble

n_models = 15
ensemble_model = small_stacking_ensemble(models_new[0:n_models], model_input, n_models)
_ = compile_and_train(ensemble_model, num_epochs=10)
accuracy = evaluate_test_accuracy(ensemble_model)
print(n_models, "test_acc:", accuracy)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 62s 2ms/step - loss: 3.5595 - acc: 0.7187 - val_loss: 1.6440 - val_acc: 0.9764
Epoch 2/10
40000/40000 [==============================] - 60s 1ms/step - loss: 0.6165 - acc: 0.9894 - val_loss: 0.2347 - val_acc: 0.9910
Epoch 3/10
40000/40000 [==============================] - 60s 2ms/step - loss: 0.1448 - acc: 0.9938 - val_loss: 0.1136 - val_acc: 0.9909
Epoch 4/10
40000/40000 [==============================] - 60s 1ms/step - loss: 0.0789 - acc: 0.9944 - val_loss: 0.0783 - val_acc: 0.9912
Epoch 5/10
40000/40000 [==============================] - 60s 1ms/step - loss: 0.0548 - acc: 0.9951 - val_loss: 0.0624 - val_acc: 0.9913
Epoch 6/10
40000/40000 [==============================] - 60s 1ms/step - loss: 0.0418 - acc: 0.9954 - val_loss: 0.0538 - val_acc: 0.9911
Epoch 7/10
 2560/40000 [>.............................] - ETA: 43s - loss: 0.0311 - acc: 0.9973

KeyboardInterrupt: 

In [17]:
# Evaluate single ensemble
ensemble_model = small_stacking_ensemble(models_new[0:n_models], model_input, n_models)
ensemble_model.load_weights('weights/CIFAR100/CIFAR100_small_stacking_15_ensemble.05-0.99.hdf5')
print(evaluate_test_accuracy(ensemble_model))


0.4863


In [11]:
# Train ensemble

basic_stacking_accuracies = np.zeros((51,1))

range_list = np.arange(0,50,5)
range_list[0] = 2

for i in range_list:
    n_models = i
    #print(n_models)
    #models_short = models_new[0:n_models]
    ensemble_model = basic_stacking_ensemble(models_new[0:n_models], model_input, i)
    _ = compile_and_train(ensemble_model, num_epochs=2)
    basic_stacking_accuracies[i] = evaluate_test_accuracy(ensemble_model)
    print(i,"test_acc:", basic_stacking_accuracies[i])

#n_models = 25

#ensemble_model = basic_stacking_ensemble(models_new[0:n_models], model_input)
#print(ensemble_model.summary())
#_ = compile_and_train(ensemble_model, num_epochs=20)

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 12s 290us/step - loss: 4.4440 - acc: 0.5019 - val_loss: 4.2885 - val_acc: 0.7864
Epoch 2/2
40000/40000 [==============================] - 10s 249us/step - loss: 4.1298 - acc: 0.8396 - val_loss: 3.9930 - val_acc: 0.8390
2 test_acc: [0.3957]
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 22s 558us/step - loss: 4.2051 - acc: 0.7007 - val_loss: 3.8214 - val_acc: 0.9314
Epoch 2/2
40000/40000 [==============================] - 21s 526us/step - loss: 3.4483 - acc: 0.9526 - val_loss: 3.1300 - val_acc: 0.9449
5 test_acc: [0.4589]
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 41s 1ms/step - loss: 3.8227 - acc: 0.8084 - val_loss: 3.0994 - val_acc: 0.9756
Epoch 2/2
40000/40000 [==============================] - 40s 997us/step - loss: 2.4511 - acc: 0.9867 - val_loss: 1.9552 

In [ ]:
# Evaluate ensemble

ensemble_model = basic_stacking_ensemble(models_new[0:n_models], model_input)
ensemble_model.load_weights('weights/CIFAR10/CIFAR10_basic_stacking_ensemble.01-0.99.hdf5')
#print(ensemble_model.summary())
print(evaluate_test_accuracy(ensemble_model))

#for model in models:
#        print(evaluate_test_accuracy(model))